In [1]:
import sys
sys.path.append(r"../")
#sys.path.append(r"../utils/data_manipulation")
import pandas as pd
from source.utils.data_manipulation.data_imputation import impute_from_column
from source.utils.consts.pathology_variables import pathology_variables_times
from source.utils.target_variable import TargetVariable
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pingouin as pg
import plotly.graph_objs as go
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import os

from statsmodels.stats.contingency_tables import mcnemar
import statsmodels.api as sm
from statsmodels.stats.contingency_tables import Table


import matplotlib
%matplotlib inline

In [15]:
df = pd.read_csv(r"../source/projects/app_data_analysis/data/DeppClinic_patient_data_treatment.csv")
df = df[df['group'].isin(['ipt', 'control'])]

df["intake_date"] =  pd.to_datetime(df["sciafca_timestamp"], errors='coerce')
df = df[df.group == 'ipt']
print(df.measurement.unique())
print(df.group.unique())

['Time 1' 'Time 2' 'Time 3']
['ipt']


In [16]:
app_ids = pd.read_csv(r"../Data/helper_docs/APP_patient_ids.csv")
def used_app(x, app_ids):
    if x['id'] in list(app_ids['patient_id']):
        return True
    else:
        return False
df['used_app'] = df.apply(used_app, args=[app_ids], axis=1)

In [17]:
intake_target_variables = list(pathology_variables_times['intake'].keys()) + ['c_ssrs_intake_life_stb']


time2_target_variables = list(pathology_variables_times['time2'].keys()) + ['c_ssrs_stb_score']


target_variables_per_time = {
    'Time 1': intake_target_variables,
    'Time 2': time2_target_variables,
    'Time 3': time2_target_variables}

intake_to_time2_map = {
 'c_ssrs_intake_life_stb': 'c_ssrs_stb_score',
 'suicidal_behavior_intake': 'suicidal_behavior_time2',
 'nssi_intake': 'nssi_time2',
 'suicidal_ideation_life_intake': 'suicidal_ideation_time2',
 'ER_intake': 'ER_time2',
 'Psychiatric_hospitalization_intake': 'Psychiatric_hospitalization_time2'
 
}

In [18]:

time1_df = df[df.measurement.isin(['Time 1'])]
time1_df = time1_df.drop(time2_target_variables, axis=1)
time1_df = time1_df.rename(intake_to_time2_map, axis=1)

time2_df = df[df.measurement.isin(['Time 2'])]
time3_df = df[df.measurement.isin(['Time 3'])]


In [19]:
#df_short.groupby(['measurement', 'used_app', 'suicidal_behavior_time2']).id.nunique()

In [20]:
df_short = pd.concat([time1_df, time2_df])
df_long = pd.concat([time1_df, time3_df])

In [21]:
current_target_vars = ['suicidal_ideation_time2', 'suicidal_behavior_time2', 'nssi_time2', 'c_ssrs_stb_score']

info_cols = ['group', 'id', 'used_app']

# df_long[current_target_vars + info_cols].to_csv(f"output/data/long_effect_raw_data.csv", index=False)
# df_short[current_target_vars + info_cols].to_csv(f"output/data/short_effect_raw_data.csv", index=False)


# Intake distributation

In [22]:
def plot (df, target, stat='anova'):
    # remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
    df = df.reset_index().drop('index', axis=1, errors='ignore')
    
    
    df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers
    
    if stat == 'anova':
        anova_result = pg.anova(data=df, dv=target, between='group')[['Source', 'F', 'p-unc']]
        anova_str = anova_result.round(decimals=3).to_string(index=False).split('\n')
        stat_text = f"<b>ANOVA Result:</b><br>{anova_str[0]}</b><br>{anova_str[1]}"
    elif stat == 'chi_square':
        expected, observed, stats = pg.chi2_independence(data=df, x=target, y='group')
        stats = stats[stats.test == 'pearson'].round(3)[['pval', 'power']].to_string(index=False).split('\n')
        stat_text = f"<b>chi_square Result:</b><br>{stats[0]}</b><br>{stats[1]}"
        
        
    chart_data = pd.concat([
        pd.Series(df.index, index=df.index, name='__index__'),
        df['group'],
        df[target],
    ], axis=1)
    chart_data = chart_data.query(f"""(`{target}` == 1) or (`{target}` == 0)""")
    chart_data = chart_data.sort_values([target, 'group'])
    chart_data = chart_data.rename(columns={'group': 'x'})
    chart_data_pctct = chart_data.groupby([target, 'x'])[['__index__']].count()
    chart_data_pctct = chart_data_pctct / chart_data_pctct.groupby(['x']).count()
    chart_data_pctct.columns = ['__index__|pctct']
    chart_data = chart_data_pctct.reset_index()
    chart_data = chart_data.dropna()
    
    chart_data = chart_data.query(f"""{target} == 1""")


    charts = []
    charts.append(go.Bar(
        x=chart_data['x'],
        y=chart_data['__index__|pctct'],
        name=f"({target.replace('_time2', '')}: 1)",
        marker_color = 'red'
    ))


    chart_data = pd.concat([
        pd.Series(df.index, index=df.index, name='__index__'),
        df['group'],
        df[target],
    ], axis=1)
    chart_data = chart_data.query(f"""(`{target}` == 1) or (`{target}` == 0)""")
    chart_data = chart_data.sort_values([target, 'group'])
    chart_data = chart_data.rename(columns={'group': 'x'})
    chart_data_pctct = chart_data.groupby([target, 'x'])[['__index__']].count()
    chart_data_pctct = chart_data_pctct / chart_data_pctct.groupby(['x']).count()
    chart_data_pctct.columns = ['__index__|pctct']
    chart_data = chart_data_pctct.reset_index()
    chart_data = chart_data.dropna()
    # WARNING: This is not taking into account grouping of any kind, please apply filter associated with
    #          the group in question in order to replicate chart. For this we're using '"""`gender` == 'man'"""'
    chart_data = chart_data.query(f"""`{target}` == 0""")

    charts.append(go.Bar(
        x=chart_data['x'],
        y=chart_data['__index__|pctct'],
        name=f"({target.replace('_time2', '')}: 0)",
        marker_color = 'green'
    ))

    figure = go.Figure(data=charts, layout=go.Layout({
        'barmode': 'group',
        'legend': {'orientation': 'h'},
        'title': {'text': f"Rate of {target.replace('_time2', '')} by Treatment Group"},
        'xaxis': {'tickformat': '0:g', 'title': {'text': 'group'}},
        'yaxis': {'tickformat': '0:g', 'title': {'text': 'Count'}, 'type': 'linear'},
    }))
    figure.add_annotation(
        x=1,
        y=1,
        text=stat_text,
        showarrow=False,
        font=dict(size=11, color='black'),
        bgcolor='lightgray',
        bordercolor='black',
        borderwidth=1,
        borderpad=12,
        xref='paper',
        yref='paper'
    )
    figure.show()
    return figure


In [23]:



df_intake = time1_df
df_intake = df_intake[current_target_vars + ['group', 'age_child_pre']]


In [24]:
#df_intake.to_csv(f'output/data/intake_groups_targets_data.csv')


for target in current_target_vars:

    figure = plot(df_intake, target, stat='chi_square')
    #figure.write_html(f"output/plots/intake_{target} X groups.html")


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\pingouin\contingency.py:150: UserWarning:

Low count on observed frequencies.

C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\pingouin\contingency.py:150: UserWarning:

Low count on expected frequencies.



# treatment improvement analysis

In [25]:
def plot_treatment_improvement (df, target, time="Time 3"):

    # remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
    df = df.reset_index().drop('index', axis=1, errors='ignore')
    df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

    chart_data = pd.concat([
        df['used_app'],
        df[target],
        df['time'],
    ], axis=1)
    chart_data = chart_data.query(f"""(`time` == 'Time 1') or (`time` == '{time}')""")
    chart_data = chart_data.sort_values(['time', 'used_app'])
    chart_data = chart_data.rename(columns={'used_app': 'x'})
    chart_data_mean = chart_data.groupby(['time','x'], dropna=True)[[target]].mean()
    chart_data_mean.columns = [f'{target}||mean']
    chart_data = chart_data_mean.reset_index()
    chart_data = chart_data.dropna()
    # WARNING: This is not taking into account grouping of any kind, please apply filter associated with
    #          the group in question in order to replicate chart. For this we're using '"""`time` == 'intake'"""'

    import plotly.graph_objs as go

    charts = []

    intake_chart_data = chart_data.query("""`time` == 'Time 1'""")
    charts.append(go.Bar(
        x=intake_chart_data['x'],
        y=intake_chart_data[f'{target}||mean'],
        name='(time: intake)'
    ))
    
    
    time3_chart_data = chart_data.query(f"""`time` == '{time}'""")
    charts.append(go.Bar(
        x=time3_chart_data['x'],
        y=time3_chart_data[f'{target}||mean'],
        name='(time: follow up)'
    ))
    
    figure = go.Figure(data=charts, layout=go.Layout({
        'barmode': 'group',
        'legend': {'orientation': 'h', 'y': -0.3},
        'title': {'text': f"Rate of {target.replace('_time2', '')} by Used App"},
        'xaxis': {'title': {'text': 'group'}},
        'yaxis': {'title': {'text': f'Rate of {target}'}, 'type': 'linear'}
    }))

    figure.show()
    return figure


In [26]:
def logistic_regresstion_test(df, target):
    X = df[['time', 'used_app']]
    Y = df[target]

    label_encoder_of_time = LabelEncoder()
    label_encoder_of_group = LabelEncoder()
    standard_scaler = StandardScaler()
    
    X['used_app'] = label_encoder_of_group.fit_transform(X['used_app'])
    X['time'] = label_encoder_of_group.fit_transform(X['time'])
    X[['used_app', 'time']] = standard_scaler.fit_transform(X[['used_app', 'time']])
    X['interaction'] = X['time'] * X['used_app']
    
    model = sm.Logit(Y, X).fit()
    
    wald_test_with_interation = model.wald_test('time + used_app + interaction = 0')
    wald_test_linear = model.wald_test('time + used_app = 0')
    
    print(f"{wald_test_with_interation = }")
    print(f"{wald_test_linear = }")



# plots - short effect

In [27]:
for target in ['suicidal_ideation_time2', 'suicidal_behavior_time2', 'nssi_time2']:
    figure = plot_treatment_improvement (df_short.rename({'measurement': 'time'}, axis=1), target, time = "Time 2")
    #figure.write_html(f"output/plots/treatment_short_effects_{target}.html")



# plots long effect

In [15]:
for target in ['suicidal_ideation_time2', 'suicidal_behavior_time2', 'nssi_time2']:
    figure = plot_treatment_improvement (df_long.rename({'measurement': 'time'}, axis=1), target)
    figure.write_html(f"output/plots/treatment_long_effects_{target}.html")



FileNotFoundError: [Errno 2] No such file or directory: 'output\\plots\\treatment_long_effects_suicidal_ideation_time2.html'